# Generating Bug Free Leetcode Solutions

!!! note
    To download this tutorial as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/bug_free_python_code.ipynb).

In this example, we want to solve String Maniuplation leetcode problems such that the code is bug free.

We make the assumption that:

1. We don't need any external libraries that are not already installed in the environment.
2. We are able to execute the code in the environment.

## Objective

We want to generate bug-free code for solving leetcode problems. In this example, we don't account for semantic bugs, only for syntactic bugs.

In short, we want to make sure that the code can be executed without any errors.

## Step 1: Generating `RAIL` Spec

First, we install the validators and packages we need to make sure generated python is valid. Then, we generate a RAIL spec as a pydantic model.

In [12]:
!guardrails hub install hub://reflex/valid_python --quiet

Installing hub://reflex/valid_python...
✅Successfully installed reflex/valid_python!




In [13]:
from pydantic import BaseModel, Field
from guardrails.hub import ValidPython

prompt = """
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
${leetcode_problem}

${gr.complete_xml_suffix}"""

class BugFreePythonCode(BaseModel):
    python_code: str = Field(validators=[ValidPython(on_fail="reask")])

    class Config:
        arbitrary_types_allowed = True

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the generated code. This object:

1. Enforces the quality criteria specified in the RAIL spec (i.e. bug free code).
2. Takes corrective action when the quality criteria are not met (i.e. reasking the LLM).
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [14]:
import guardrails as gd

from rich import print

Or from the pydantic model:

In [15]:
guard = gd.Guard.from_pydantic(output_class=BugFreePythonCode)

## Step 3: Wrap the LLM API call with `Guard`

In [16]:
import openai

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

response = guard(
    openai.chat.completions.create,
    prompt=prompt,
    prompt_params={"leetcode_problem": leetcode_problem},
    max_tokens=2048,
    temperature=0,
)

PromptCallableException: The callable `fn` passed to `Guard(fn, ...)` failed with the following error: `Connection error.`. Make sure that `fn` can be called as a function that takes in a single prompt string and returns a string.

The `Guard` object compiles the output schema and adds it to the prompt. We can see the final prompt below:

In [ ]:
print(guard.history.last.prompt)

AttributeError: 'Guard' object has no attribute 'rail'

Running the cell above returns:
1. The raw LLM text output as a single string.
2. A dictionary where the key is `python_code` and the value is the generated code.

In [ ]:
print(response.validated_output)

{
    'python_code': 'def longest_palindromic_substring(s):\n    if not s:\n        return ""\n    start = 0\n    end
= 0\n    for i in range(len(s)):\n        len1 = expand_around_center(s, i, i)\n        len2 = 
expand_around_center(s, i, i + 1)\n        max_len = max(len1, len2)\n        if max_len > end - start:\n          
start = i - (max_len - 1) // 2\n            end = i + max_len // 2\n    return s[start:end + 1]\n\n\ndef 
expand_around_center(s, left, right):\n    while left >= 0 and right < len(s) and s[left] == s[right]:\n        
left -= 1\n        right += 1\n    return right - left - 1'
}

Here's the generated code:

In [ ]:
if response.validated_output is not None:
    print(response.validated_output["python_code"])
elif response.error is not None:
    print(response.error)

def longest_palindromic_substring(s):
    if not s:
        return ""
    start = 0
    end = 0
    for i in range(len(s)):
        len1 = expand_around_center(s, i, i)
        len2 = expand_around_center(s, i, i + 1)
        max_len = max(len1, len2)
        if max_len > end - start:
            start = i - (max_len - 1) // 2
            end = i + max_len // 2
    return s


def expand_around_center(s, left, right):
    while left >= 0 and right < len(s) and s == s:
        left -= 1
        right += 1
    return right - left - 1

We can confirm that the code is bug free by executing the code in the environment.

In [ ]:
try:
    exec(response.validated_output["python_code"])
    print("Success!")
except Exception as e:
    print("Failed!")

Success!